<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/modern_approach/text_to_image/stable_diffusion_variations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stable Diffusion variations

In [ ]:
!git clone https://huggingface.co/spaces/lambdalabs/stable-diffusion-image-variations
%cd stable-diffusion-image-variations/

In [ ]:
!pip install -r requirements.txt
!pip install -q gradio

In [ ]:
import gradio as gr
import torch
from PIL import Image
from lambda_diffusers import StableDiffusionImageEmbedPipeline

device = 'cuda'
pipe = StableDiffusionImageEmbedPipeline.from_pretrained(
    "lambdalabs/sd-image-variations-diffusers",
    revision="273115e88df42350019ef4d628265b8c29ef4af5"
)
pipe = pipe.to(device)

def main(input, scale=3.0, n_samples=4, steps=25, seed=0):

  generator = torch.Generator(device=device).manual_seed(int(seed))

  images_list = pipe(
      n_samples * [input],
      guidance_scale=scale,
      n_inference_steps=steps,
      generator=generator
  )
  images = []
  for i, img in enumerate(images_list['sample']):
    if images_list['nsfw_content_detected'][i]:
      safe_img = Image.open(r'unsafe.png')
      images.append(safe_img)
    else:
      images.append(img)
  
  return images

inputs = [
    gr.Image(),
    gr.Slider(0, 25, value=3, step=1, label='Guidance scale'),
    gr.Slider(1, 4, value=1, step=1, label='Number images'),
    gr.Slider(5, 50, value=25, step=5, label='Steps'),
    gr.Number(0, label='Seed', precision=0)
]
output = gr.Gallery(label='Generated variations')
output.style(grid=2)

demo = gr.Interface(
    fn=main, 
    title='Image variations',
    inputs=inputs,
    outputs=output
)
demo.launch(debug=True)